## let's check if we bought the brazilian index after it closed  below the 2 times std(20 periods)  
## in intraday operations and performed better than if we didn't use this strategy 



In [137]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import timedelta
from scipy import stats 
from scipy.stats import ttest_ind


In [138]:

path = "C:\\Users\\muril\\OneDrive\\Área de Trabalho\\analisis\\indice\\"


df = pd.read_csv(path+'diario_ind.csv', encoding='latin-1',sep=';')

In [89]:
df.head()

,Data,Abertura,Máxima,Mínima,Fechamento
0,13/10/2023,116300,117185,115685,115875
1,11/10/2023,117295,117695,116385,117405
2,10/10/2023,115900,117180,115745,116760
3,09/10/2023,113620,115695,113440,115690
4,06/10/2023,113800,114795,111860,114700


In [90]:
df["Data"] = pd.to_datetime(df["Data"])
df.set_index("Data", inplace=True)

C:\Users\muril\AppData\Local\Temp\ipykernel_19320\4018039805.py:1: UserWarning:

Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.



In [91]:
# some colums has "," in the data, lets clean this and transforme these numbers
for colum in df.columns:
    df[colum] = df[colum].apply(lambda x : str(x).split(",")[0])
    df[colum] = pd.to_numeric(df[colum])

In [92]:
df.sort_values("Data", inplace=True)

In [93]:
# some usefull columns
df["pct"] = df["Fechamento"].pct_change()
df["log"] = np.log(df["Fechamento"]).diff()    
df["std"] = np.std(df["log"])
df["std_mean"] = df["log"].rolling(20).std()
df["mean"] =   df["log"].rolling(window=20).mean()

In [94]:
fig = go.Figure()
fig.add_trace(go.Scatter(name = "Log", x=df.index, y = df["log"]))
fig.add_trace(go.Scatter(name = "Std", x=df.index, y = df["std"]))
fig.add_trace(go.Scatter(name = "Std_baixo", x=df.index, y = df["std"]*-1))
fig.add_trace(go.Scatter(name = "Std_mean", x=df.index, y = df["std_mean"]))
fig.add_trace(go.Scatter(name = "Std_mean_baixo", x=df.index, y = df["std_mean"]*-2))
fig.add_trace(go.Scatter(name = "Media", x=df.index, y = df["mean"]))
  


In [124]:
# select a data frame that meets my requirements  
check_compra = df[df["log"] < df["std_mean"]*-2]
check_compra.tail()

,Abertura,Máxima,Mínima,Fechamento,pct,log,std,std_mean,mean
Data,,,,,,,,,
2022-08-19,128702,129027,126296,126318,-0.027994,-0.028393,0.017794,0.012619,0.005146
2023-01-02,118797,118996,115424,115569,-0.032734,-0.033281,0.017794,0.016491,-0.003309
2023-03-23,106749,107835,103237,103359,-0.030994,-0.031485,0.017794,0.011807,-0.005627
2023-07-27,126122,126346,122486,122618,-0.023991,-0.024283,0.017794,0.011166,-0.000226
2023-09-21,118700,118715,116805,116885,-0.022128,-0.022377,0.017794,0.010198,-0.001498


In [125]:

df.dropna(inplace=True)

In [126]:
# let's find the row that satisfies my criteria and take the next day. my focus is to check if the day after  the satisfied day
# is a good day to trade in buy side. 

values = []
for idx, row  in check_compra.iterrows():
    index_novo = df.index.get_loc(idx)+1
    values.append(df.iloc[index_novo])
    

In [128]:
trades_ = pd.DataFrame(values,columns =df.columns)
mask = df.index.isin(trades_.index)
df_analise = df[~mask]
df_analise.dropna(inplace=True)

C:\Users\muril\AppData\Local\Temp\ipykernel_19320\4263498143.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [130]:
t_stat, p_value = ttest_ind(trades_["pct"], df_analise["pct"])
t_stat2, p_value2 = ttest_ind(trades_["pct"], df["pct"])
print("The p_value rejects the H0 hypothesis that there is no statistical difference between first df and third data frame\n p-value {}".format(p_value))
print("The p_value rejects the H0 hypothesis that there is no statistical difference between first df and second data frame\n p-value {}".format(p_value2))





The p_value rejects the H0 hypothesis that there is no statistical difference between first df and third data frame
 p-value 3.766720065098287e-05
The p_value rejects the H0 hypothesis that there is no statistical difference between first df and second data frame
 p-value 7.639250811342508e-05


In [132]:
print("The first has an average return {}\nThe Second has an average return{}\nThe Third has an average return {}".format(trades_["pct"].mean(),df["pct"].mean(),df_analise["pct"].mean()))





The first has an average return 0.006777216944436978
The Second has an average return0.00018861400375632604
The Third has an average return 9.697426955765224e-06


In [134]:
firt_cum = (1+trades_["pct"]).cumprod()[-1]

In [135]:
second_cum = (1+df["pct"]).cumprod()[-1]
third_cum = (1+df_analise["pct"]).cumprod()[-1]

In [136]:
print("The first has an cumulative return {}\nThe Secondhas an cumulative return{}\nThe Third has an cumulative return {}".format(firt_cum,second_cum,third_cum))


The first has an cumulative return 2.1661327306449194
The Secondhas an cumulative return1.1484939490351231
The Third has an cumulative return 0.5302047897559736


## In conclusion, by analyzing the possibility of making random purchases every day on the Brazilian index or buying only when the test condition is met (closing below the 2 times 20-day average standard deviation) we can conclude that your performance will be much higher following this premise, as proven by the Ttest, average returns and accumulated returns. 